In [1]:
%load_ext autoreload
%autoreload 2

In [185]:
from ampl_utils import ampl_license
from amplpy import AMPL, ampl_notebook

from models import MixedStrategyMasterProblem

from dataloader import Network

solver='cplex'
ampl_license_uuid = ampl_license()

ampl = ampl_notebook(
    modules=[solver],
    license_uuid=ampl_license_uuid)

Licensed to AMPL Academic Community Edition License for <s207399@student.pg.edu.pl>.


### Master Problem

$$
\begin{aligned}
\mathbb P[\mathcal S, \mathcal A]: \quad & \max y \quad & \\
[x] \quad & \sum_{s\in\mathcal S}q_s = 1 \quad & \\
[p_a\geq 0]\quad & y \leq \sum_{s\in\mathcal S}V(s, a)q_s \quad & a\in\mathcal A \\
 & y \in\mathbb R; q_s\in\mathbb{R}_+\quad & s\in\mathcal S
\end{aligned}
$$

Here's a simple test of the master problem

In [190]:
network = Network('dognet')
attack_set = [
    [1, 2, 3], [5, 7, 8], [7, 2, 3], [4, 9, 8], [1, 5, 4], [9, 2, 1]
]

placement_set = [
    [1, 9], [5, 4], [2, 7], [3, 8], [8, 9]
]
mp = MixedStrategyMasterProblem(network, placement_set, attack_set)
mp.solve()

CPLEX 22.1.2: optimal solution; objective 3.25
6 simplex iterations


### Attack Generation Pricing Problem

The attack generation pricing problem is denoted by

$$
\begin{aligned}
\min\sum_{s\in\mathcal S'}q^*_s\cdot F_s \quad & \quad &  \\
\sum_{v\in\mathcal V} a_v = K \quad&\quad & \text{Must be a $K$-node attack}\\
z_v^s = 1-a_v \quad & v\in\mathcal V, s\in\mathcal S' \quad& \text{$v$ survives attack $a$ given $s$} \\
z_{\beta(e)}^s \geq z_{\alpha(e)^s} - a_{\beta(e)} \quad & e\in\mathcal E, s\in\mathcal S'\quad & \\
z_{\alpha(e)}^s \geq z_{\beta(e)^s} - a_{\alpha(e)} \quad & e\in\mathcal E, s\in\mathcal S'\quad & \\
F_s = \sum_{v\in\mathcal V}z_v^s \quad & s\in\mathcal S' \quad & \text{The number of surviving nodes per placement} \\
a_v, z_v^s\in\mathbb B\quad & s\in\mathcal S', v\in\mathcal V \quad & \\
F_s \in\mathbb Z_+ \quad & s\in\mathcal S' \quad & 
\end{aligned}
$$

where $\mathcal S'=\{s\in\mathcal S : q^*(s) > 0 \}$ is the set of placements with non-zero probabilities of being used.

In [ ]:
%%writefile models/attack_generation_pricing_problem.mod

set V; # The number of nodes
set S_PRIME; # Mon-zero probability placements
set E; # The number of edges

var F{S_PRIME} integer >= 1;
var a{V} binary;
var z{S_PRIME, V} binary;

# Payoff of the network operator
param q_star{S_PRIME};
# Total required attacked nodes.
param K;

min ControllerPayoff:
    sum {s in S_PRIME} (q_star[s] * F[s]);

s.t. ConstraintAttackSize:
    sum {v in V} a[v] = K;

s.t. ConstraintNodeSurvival {v in V, s in S_PRIME}:
    z[s, v] = 1 - a[v];

# s.t. ConstraintNeighborhoodSurvival {e in EDGES